In [42]:
#Opencv Python - Similarity score from Feature matching + Homograpy
#https://answers.opencv.org/question/60350/opencv-python-similarity-score-from-feature-matching-homograpy/
#Homography : http://www.gisdeveloper.co.kr/?p=6832

import cv2
import numpy as np
import os

In [52]:
def feature_matcher(query_image, image_folder):

    min_match_count = 10

    img1 = cv2.imread(query_image, 0)
    surf = cv2.xfeatures2d.SURF_create(800)
    kp1, des1 = surf.detectAndCompute(img1, None)

    bf = cv2.BFMatcher(cv2.NORM_L2)

    path_files = os.getcwd()
    all_files = next(os.walk('.'+image_folder))[2]
#    print(path_files + all_files[0])

    good_matches = []
    match_max = -1
    match_img = ''
    
    for file_name_temp in all_files:
        try:
            train_image = path_files + image_folder + '/' + file_name_temp
#            print(train_image)
            
#             t = cv2.imread(train_image, cv2.IMREAD_COLOR)
#             cv2.imshow('img1', img1)
#             cv2.imshow('test', t)
#             cv2.waitKey(0)
#             cv2.destroyAllWindows()
#             cv2.waitKey(1)
            
            img2 = cv2.imread(train_image, 0)
            surf = cv2.xfeatures2d.SURF_create(800)
            kp2, des2 = surf.detectAndCompute(img2, None)

#             print(1)
            matches = bf.knnMatch(des1, des2, k=2)

#             print(2)
            good = []
            for m, n in matches:
                if m.distance < 0.7*n.distance:
                    good.append(m)
                    
#             print(3)
            if len(good) > min_match_count:

                src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
                dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
                
#                 print(4)
                M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

#                 print(5)
                matchesMask = mask.ravel().tolist()

#                 print(6)
                h, w = img1.shape
                pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
                dst = cv2.perspectiveTransform(pts, M)

#                 print(7)
#                 if not M==None:
#                     print(8)
#                     print()
#                     print("-" * 2, file_name_temp)
#                     print("number of good matches", len(good))
#                     print("*" * 10, matchesMask)

                #Get matches avg, max, min
                good_matches.append(len(good))
    
                print("-" * 2, file_name_temp)
                print("number of good matches", len(good))
                
                #Get max matches img, len(good)
                if match_max < len(good):
                    match_max = len(good)
                    match_img = file_name_temp
                
#                 print("*" * 10, matchesMask)              
#                 print(type(matchesMask))
#                 np_matchedMask = np.array(matchesMask, dtype=np.bool)
#                 print(type(np_matchedMask))
#                 print(np.count_nonzero(np_matchedMask), len(np_matchedMask))
                print()
#                 print(9)
            
        except Exception as ex: # 에러 종류
            print('에러가 발생 했습니다', ex)
            
    print(good_matches)
    print(max(good_matches))
    print(min(good_matches))
#     print(match_max)
    print(match_img)
#     print(good_matches.index(max(good_matches)))
#     print(all_files[good_matches.index(max(good_matches))+1])
    print(1.0 * sum(good_matches) / len(good_matches))
                    

In [53]:
# query_image = '/home/zealous/Pictures/train_images/AbudefdufWhitleyiJER.jpg'
# trained_image_folder = '/home/zealous/Pictures/train_images'

#query_image = './image/test/img0.png'
# query_image = './image/test/img8.png'
query_image = './image/test/t_img_1.jpeg'
image_folder = '/image/train'
feature_matcher(query_image, image_folder)

-- img36.png
number of good matches 84

-- img22.png
number of good matches 101

-- img119.png
number of good matches 304

-- img125.png
number of good matches 356

-- img131.png
number of good matches 150

-- img253.png
number of good matches 203

-- img247.png
number of good matches 224

-- img290.png
number of good matches 260

-- img284.png
number of good matches 203

-- img285.png
number of good matches 158

-- img291.png
number of good matches 111

-- img246.png
number of good matches 193

-- img252.png
number of good matches 185

-- img130.png
number of good matches 304

-- img124.png
number of good matches 250

-- img118.png
number of good matches 265

-- img23.png
number of good matches 61

-- img37.png
number of good matches 92

-- img21.png
number of good matches 45

-- img35.png
number of good matches 114

-- img132.png
number of good matches 153

-- img126.png
number of good matches 356

-- img278.png
number of good matches 186

-- img244.png
number of good matches 165

--